In [ ]:
!pip install pandas
!pip install html2text
!pip install schedule
!pip install sinling 
!pip install regex
!pip install deep_translator

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 40.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 353.7/353.7 kB 23.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 993.5/993.5 kB 48.7 MB/s eta 0:00:00
  Created wheel for emoji: filename=emoji-2.4.0-py2.py3-none-any.whl size=350809 sha256=86d3baabac56cc8869e7ac747c601fd25ffe20c8091c8b7ec825103e57910e89
  Stored in directory: /root/.cache/pip/wheels/79/29/1c/234cae4632803c2ba4a76a71a679eb1383cf590775714e2a21
Successfully built emoji
Looking in indexes: https://pypi.or

In [ ]:
import requests
from bs4 import BeautifulSoup
import html2text
from urllib.parse import urljoin
import os
import re
import schedule
import time
from datetime import datetime, timedelta
import random
from sinling import SinhalaTokenizer
from deep_translator import GoogleTranslator
import pandas as pd
import csv

# List of invalid URLs to skip
invalid_urls = [
    'tg://', 'tel:', 'qz', 'ru', 'en', 'cn', 'oz', 'ar'
]

# Set to store unique links
unique_links = set()

# Set to store processed links
processed_links = set()

# Configure HTML to text converter
converter = html2text.HTML2Text()
converter.ignore_links = True
converter.ignore_images = True
converter.single_line_break = True

def main(url, output_file):
    """
    Main function to scrape and process content from a URL
    """
    url_parts = url.split('/')
    if any(x in url_parts for x in invalid_urls):
        print(f'Skipping invalid URL: {url}')
        return

    try:
        response = requests.get(url)
    except requests.exceptions.SSLError as e:
        print(f'SSLError fetching {url}')
        return

    with open(output_file, 'a', encoding='utf-8') as f:
        # Fetch the main page content
        main_page_response = requests.get(url)
        main_page_content = main_page_response.text

        # Parse the main page and extract unique links
        soup = BeautifulSoup(main_page_content, 'html.parser')

        for link in soup.find_all('a', href=True):
            link_url = link['href']
            absolute_link_url = urljoin(url, link_url)
            url_parts = absolute_link_url.split('/')
            if not any(x in url_parts for x in invalid_urls):
                if absolute_link_url not in unique_links:
                    unique_links.add(absolute_link_url)

        # Scrape the content of the main page
        try:
            plain_text_content = converter.handle(main_page_content)
            f.write(plain_text_content)
        except Exception as e:
            print(f'Error fetching main page {url}: {e}')

        # Scrape the content of each linked page
        for link_url in unique_links:
            if link_url not in processed_links:
                try:
                    response = requests.get(link_url)
                    content = response.text
                    plain_text_content = converter.handle(content)
                    f.write(plain_text_content)
                    processed_links.add(link_url)
                except requests.exceptions.SSLError as e:
                    print(f'SSLError fetching {link_url}: {e}')
                    continue
                except Exception as e:
                    print(f'Error fetching {link_url}: {e}')
                    continue

def tokenize_sinhala(text):
    """
    Tokenize Sinhala text into sentences
    """
    sinhala_tokenizer = SinhalaTokenizer()
    sentences = sinhala_tokenizer.split_sentences(text)
    return sentences

def create_parallel_corpus(input_file, output_file):
    """
    Create a parallel corpus from an input file containing Sinhala text
    """
    with open(input_file, 'r', encoding='utf-8') as f:
        text = f.read()

    sinhala_sentences = tokenize_sinhala(text)

    with open(output_file, 'a', encoding='utf-8') as f:
        for sentence in sinhala_sentences:
            f.write(sentence + '\n')

    df = pd.DataFrame(sinhala_sentences, columns=["sinhala_sentence"])
    df.to_csv(output_file, index=False, encoding='utf-8')

def translate_text(text, target_lang):
    """
    Translate Sinhala text to the target language
    """
    sentences = tokenize_sinhala(text)
    split_sentences = []
    for sentence in sentences:
        if len(sentence) >= 5000:
            print(f"Skipping sentence with length {len(sentence)}: {sentence}")
            with open('split_sentences.csv', 'a') as f:
                f.write(sentence + '\n')
            continue

        split_sentences.append(sentence)

    translator = GoogleTranslator(target=target_lang)
    translated_sentences = []
    max_retries = 5
    backoff_factor = 2
    for sentence in split_sentences:
        if len(sentence.strip()) > 10:
            for attempt in range(max_retries):
                try:
                    translation = translator.translate(sentence)
                    translated_sentences.append(translation)
                    break
                except Exception as e:
                    if attempt < max_retries - 1:
                        wait_time = backoff_factor ** (attempt + 1)
                        print(f"Error in translation, retrying in {wait_time} seconds: {e}")
                        time.sleep(wait_time)
                    else:
                        print(f"Translation failed after {max_retries} retries: {e}")
                        translated_sentences.append("[TRANSLATION_FAILED]")  # Add a placeholder for failed translations
        else:
            print(f"Skipping too short or empty sentence: {sentence}")

    return " ".join(translated_sentences)

def translate_parallel_corpus(input_file, output_file, target_lang):
    """
    Translate a parallel corpus from Sinhala to the target language
    """
    df = pd.read_csv(input_file, encoding='utf-8')

    # Add sentence_id column
    df['sentence_id'] = range(1, len(df) + 1)

    # Translate only new sentences by sentence_id 
    existing_df = pd.DataFrame()  # Initialize existing_df variable

    try: 
        existing_df = pd.read_csv(output_file, encoding='utf-8')
        existing_ids = set(existing_df.sentence_id)
        new_df = df[~df.sentence_id.isin(existing_ids)]
        translations = []
        for sinhala_sentence in new_df.sinhala_sentence:
            translation = translate_text(sinhala_sentence, target_lang)
            translations.append(translation)
        new_df['translated_sentence'] = translations
    except FileNotFoundError:
        new_df = df
        new_df['translated_sentence'] = [translate_text(s, target_lang) for s in df.sinhala_sentence] 

    # Append new translations to output file
    try:
        existing_df.to_csv(output_file, mode='a', index=False, header=False, encoding='utf-8')
        new_df.to_csv(output_file, mode='a', index=False, header=False, encoding='utf-8')
    except FileNotFoundError:
        # Output file does not exist yet, so create it 
        df.to_csv(output_file, index=False, encoding='utf-8')

def get_next_run_time():
    """
    Generate the next run time for scheduling the script
    """
    now = datetime.now()
    # Generate a random time interval between 24 to 28 hours
    random_offset = random.randint(24, 28) * 3600  # Convert hours to seconds
    next_run_time = now + timedelta(seconds=random_offset)
    return next_run_time

def execute_main():
    """
    Execute the main function for scraping and translation
    """
    print("Running main function!")
    main_urls = [
        'https://www.bbc.com/sinhala',
        'https://www.bbc.com/sinhala/topics/cg7267dz901t',
        'https://www.bbc.com/sinhala/topics/c83plvepnq1t',
        'https://www.lankadeepa.lk/latest-news/1',
        'https://www.kelimandala.lk/',
        'https://www.lankadeepa.lk/tharunaya/272',
        'https://www.lankadeepa.lk/business/9',
        'https://www.lankadeepa.lk/politics/13'
    ]

    output_files = [
        'sinhala_general_news(1).txt',
        'sinhala_srilankan_news.txt',
        'sinhala_world_news.txt',
        'sinhala_general_news(2).txt',
        'sinhala_sports.txt',
        'sinhala_general_news(3).txt',
        'sinhala_business.txt',
        'sinhala_politics.txt'
    ]

    for url, output_file in zip(main_urls, output_files):
        main(url, output_file)
        unique_links.clear()  # Clear unique_links set for each main URL

    # Translate the scraped content
    for output_file in output_files:
        parallel_corpus_file = output_file.replace('.txt', '_parallel_corpus.csv')
        create_parallel_corpus(output_file, parallel_corpus_file)
        translated_parallel_corpus_file = output_file.replace('.txt', '_translated_parallel_corpus.csv')
        translate_parallel_corpus(parallel_corpus_file, translated_parallel_corpus_file, 'en')  # Specify the target language ('en' for English)

def schedule_script():
    """
    Schedule the script to run periodically
    """
    # Run the execute_main function immediately
    execute_main()

    # Schedule the next run and continue scheduling
    while True:
        schedule.clear()  # Clear any prior schedules
        next_run_time = get_next_run_time()
        print(f"Next run at {next_run_time}")
        schedule.every(1).day.at(next_run_time.strftime('%H:%M')).do(execute_main)

        # Sleep until the next run
        time.sleep((next_run_time - datetime.now()).total_seconds())

        schedule.run_pending()  # Run any pending jobs

# Start scheduling
schedule_script()


Running main function!
Error fetching http://epaper.tamilmirror.lk/: HTTPConnectionPool(host='epaper.tamilmirror.lk', port=80): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f80cc2ebb20>: Failed to establish a new connection: [Errno -2] Name or service not known'))
Error fetching http://epaper.ada.lk/: HTTPConnectionPool(host='epaper.ada.lk', port=80): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f80cc2ebc40>: Failed to establish a new connection: [Errno -2] Name or service not known'))
Error fetching http://life.dailymirror.lk/: HTTPConnectionPool(host='life.dailymirror.lk', port=80): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f80cc2ebc10>: Failed to establish a new connection: [Errno -5] No address associated with hostname'))
Error fetching mailto:subs@wijeya.lk: No connection adapters

KeyboardInterrupt: ignored